### Importing all the libraries

## Installing all the packages required

In [1]:
!conda install -c anaconda scikit-learn
!conda install -c conda-forge folium
!conda install -c conda-forge geopy
!conda install -c conda-forge beautifulsoup4

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following packages will be UPDATED:

    libgcc-ng:    7.2.0-h7cc24e2_2      --> 8.2.0-hdf63c60_1      anaconda
    libstdcxx-ng: 7.2.0-h7a57d05_2      --> 8.2.0-hdf63c60_1      anaconda
    mkl:          2018.0.0-hb491cac_4   --> 2018.0.3-1            anaconda
    scikit-learn: 0.19.1-py35hbf1f462_0 --> 0.20.0-py35h4989274_1 anaconda

libgcc-ng-8.2. 100% |################################| Time: 0:00:00  53.81 MB/s
libstdcxx-ng-8 100% |################################| Time: 0:00:00  63.48 MB/s
mkl-2018.0.3-1 100% |################################| Time: 0:00:03  58.89 MB/s
scikit-learn-0 100% |################################| Time: 0:00:00  58.63 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will 

## Importing all the libraries

In [2]:
import pandas as pd
import requests
import numpy as np
import folium
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup

# #Getting table from Wikipedia

In [3]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
print(soup.prettify())

My_table = soup.find('table',{'class':'wikitable sortable'})
My_table

links = My_table.findAll('tr')[1:]


link2 = My_table.findAll('tr')[1:]

link3 = My_table.findAll('tr')[1:]
link2
Postcode = []
Borough = []
Neighborhood = []
for link in links:
    x = link.find_all('td')[0]
    y = link.find_all('td')[2]
    Postcode.append(x.text)
    Borough.append(y.text)
    z = link.find_all('td')[1]
    Neighborhood.append(z.text)

##    y = link.find_all('td')[1]
#    Borough.append(y.text)
    
#for link in link3:
#    z = link.find_all('a')
#    Neighborhood.append(z.text)
    


<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":890001695,"wgRevisionId":890001695,"wgArticleId":539066,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wg

## Creating a data frame from the table

In [4]:
df = pd.DataFrame(columns=['Postcode','Borough','Neighborhood'])

## Filling the pandas data frame

In [5]:
df['Postcode'] = Postcode
df['Borough'] = Neighborhood
df['Neighborhood'] = Borough

## cleaning the neighborhood coloumn

In [6]:
x = df['Neighborhood']

In [7]:
y = list(x)

In [8]:
z = [col.split('\n')[0] for col in y]

In [9]:
df['Neighborhood'] = z

## Removing all NaN values

In [10]:
df.replace('Not assigned','NaN', inplace = True)


In [11]:
df.head()

,Postcode,Borough,Neighborhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [12]:
borough_not_required = df[df['Borough']== 'NaN'].index.tolist()

In [13]:
df = df.drop(borough_not_required).reset_index(drop=True)

In [14]:
df.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


## Joining the Neighborhood coloumn for the same Postcode

In [15]:
x  = df.groupby(['Postcode','Borough'])['Neighborhood'].apply(','.join)

In [16]:
toronto = pd.DataFrame(x) 

In [17]:
toronto.reset_index(inplace=True)

In [18]:
toronto.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [19]:
toronto.shape

(103, 3)

## Task 2

In [20]:
!conda install -c conda-forge geocoder

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geocoder:   1.38.1-py_0  conda-forge
    orderedset: 2.0-py35_0   conda-forge
    ratelim:    0.1.6-py35_0 conda-forge

orderedset-2.0 100% |################################| Time: 0:00:00  13.16 MB/s
ratelim-0.1.6- 100% |################################| Time: 0:00:00  11.86 MB/s
geocoder-1.38. 100% |################################| Time: 0:00:00  39.84 MB/s


In [21]:
import geocoder

## Defining function for getting latitude longitude for postal code

In [22]:
def getlatlng(postal_code):
    lat_lng_coords = None
    while (lat_lng_coords == None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        return lat_lng_coords

In [23]:
getlatlng('M1B')

[43.811525000000074, -79.19551721399995]

In [24]:
coords = [getlatlng(x) for x in toronto['Postcode'].tolist()]

## Adding coordinates to main dataframe i.e. toronto

In [25]:
toronto['Latitude']=[coords[i][0] for i in range(len(coords))]
toronto['Longitude'] = [coords[i][1] for i in range(len(coords))] 

In [26]:
toronto.head(10)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440
5,M1J,Scarborough,Scarborough Village,43.743125,-79.231750
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.726245,-79.263670
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.713133,-79.285055
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.723575,-79.234976
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.696665,-79.260163


In [27]:
toronto[toronto['Postcode'] == 'M5G']

,Postcode,Borough,Neighborhood,Latitude,Longitude
57,M5G,Downtown Toronto,Central Bay Street,43.656091,-79.38493


## Task 3

In [28]:
from sklearn.cluster import KMeans

In [29]:
toronto.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


## SEecting Downtown Toronto borough

In [30]:
downtown_toronto = toronto[toronto['Borough']=="Downtown Toronto"]

In [31]:
downtown_toronto.reset_index(drop=True,inplace=True)

In [32]:
downtown_toronto.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.681940,-79.378474
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.668160,-79.366602
2,M4Y,Downtown Toronto,Church and Wellesley,43.666585,-79.381302
3,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.655120,-79.362640
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.657363,-79.378180


In [33]:
adress = 'Downtown Toronto'
x = Nominatim(user_agent="kartik")
geo = x.geocode(adress)

In [34]:
down_lat = geo.latitude
down_lng = geo.longitude

In [35]:
down_lat,down_lng

(43.6541737, -79.3808116451341)

In [36]:
down_map =  folium.Map([down_lat,down_lng], zoom_start = 12)

In [37]:
down_map

In [101]:
for lat , lng , neighborhood in zip(downtown_toronto['Latitude'],downtown_toronto['Longitude'],downtown_toronto['Neighborhood']):
    label = folium.Popup(neighborhood, parse_html=True)
    folium.CircleMarker([lat, lng],
                        radius=5,
                        popup=label,
                        color='blue',
                        fill=True,
                        fill_color='#3186cc',
                        fill_opacity=0.7,
                        parse_html=False).add_to(down_map)

In [40]:
CLIENT_ID = 'UUDATNQLSPQ3VLPACBR2Q2Q0BFDCM0ZXNM0D4T24R2TKN2WS' # your Foursquare ID
CLIENT_SECRET = 'ISR2D5ZTUDEZ5FHP3OEZNFB4HJCHUBFTDOVE5KHKL2OALHID' # your Foursquare Secret
VERSION = '20190421' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UUDATNQLSPQ3VLPACBR2Q2Q0BFDCM0ZXNM0D4T24R2TKN2WS
CLIENT_SECRET:ISR2D5ZTUDEZ5FHP3OEZNFB4HJCHUBFTDOVE5KHKL2OALHID


In [41]:
downtown_toronto.loc[0,'Neighborhood']

'Rosedale'

## lets get top 100 venue near Rosedale

In [42]:
neighborhood_lat = downtown_toronto.loc[0,'Latitude']
neighborhood_lng = downtown_toronto.loc[0,'Longitude']


In [43]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_lat, 
    neighborhood_lng, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=UUDATNQLSPQ3VLPACBR2Q2Q0BFDCM0ZXNM0D4T24R2TKN2WS&client_secret=ISR2D5ZTUDEZ5FHP3OEZNFB4HJCHUBFTDOVE5KHKL2OALHID&v=20190421&ll=43.681940000000054,-79.37847416699998&radius=500&limit=100'

In [44]:
results = requests.get(url).json()

In [45]:

venue =results['response']['groups'][0]['items']

In [46]:
from pandas.io.json import json_normalize

In [47]:
nearby_venue = json_normalize(venue)

In [48]:
nearby_venue['venue.categories'][0][0].keys()

dict_keys(['shortName', 'name', 'id', 'icon', 'primary', 'pluralName'])

In [49]:
def get_category(row):
    try:
        category_list = row['categories']
    except:
        category_list = row['venue.categories']
    if len(category_list) == 0:
        return None
    else:
        return category_list[0]['name']

In [50]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venue =nearby_venue.loc[:, filtered_columns]

In [51]:
nearby_venue

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,TD Canada Trust - Closed,"[{'shortName': 'Bank', 'name': 'Bank', 'id': '...",43.686094,-79.376549
1,Rosedale Park,"[{'shortName': 'Playground', 'name': 'Playgrou...",43.682328,-79.378934
2,Rosedale Tennis Club,"[{'shortName': 'Tennis Court', 'name': 'Tennis...",43.683226,-79.378984
3,Whitney Park,"[{'shortName': 'Park', 'name': 'Park', 'id': '...",43.682036,-79.373788
4,Mooredale House,"[{'shortName': 'Building', 'name': 'Building',...",43.678631,-79.380091


In [52]:
nearby_venue['venue.categories'] = nearby_venue.apply(get_category, axis=1)

In [53]:
nearby_venue

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,TD Canada Trust - Closed,Bank,43.686094,-79.376549
1,Rosedale Park,Playground,43.682328,-79.378934
2,Rosedale Tennis Club,Tennis Court,43.683226,-79.378984
3,Whitney Park,Park,43.682036,-79.373788
4,Mooredale House,Building,43.678631,-79.380091


In [54]:

downtown_toronto.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.681940,-79.378474
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.668160,-79.366602
2,M4Y,Downtown Toronto,Church and Wellesley,43.666585,-79.381302
3,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.655120,-79.362640
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.657363,-79.378180


## Explore all neighborhood near Downtown

In [55]:
def get_nearby_venue(names, latitudes, longitudes, radius=500):
    venue_list = []
    for name, lat , lng in zip(names, latitudes, longitudes):
        print(name)
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venue_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']
        ) for v in results])
    
    nearby_venues = pd.DataFrame([cols for x in venue_list for  cols in x])
    nearby_venues.columns = ['Neighborhood' , 'Neighborhood Latitude',
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)
        

In [56]:
downtown_venues = get_nearby_venue(downtown_toronto['Neighborhood'] , downtown_toronto['Latitude'] , downtown_toronto['Longitude'])

Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie


In [57]:
downtown_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.68194,-79.378474,TD Canada Trust - Closed,43.686094,-79.376549,Bank
1,Rosedale,43.68194,-79.378474,Rosedale Park,43.682328,-79.378934,Playground
2,Rosedale,43.68194,-79.378474,Rosedale Tennis Club,43.683226,-79.378984,Tennis Court
3,Rosedale,43.68194,-79.378474,Whitney Park,43.682036,-79.373788,Park
4,Rosedale,43.68194,-79.378474,Mooredale House,43.678631,-79.380091,Building


In [58]:
downtown_venues['Neighborhood']

0                                    Rosedale
1                                    Rosedale
2                                    Rosedale
3                                    Rosedale
4                                    Rosedale
5                  Cabbagetown,St. James Town
6                  Cabbagetown,St. James Town
7                  Cabbagetown,St. James Town
8                  Cabbagetown,St. James Town
9                  Cabbagetown,St. James Town
10                 Cabbagetown,St. James Town
11                 Cabbagetown,St. James Town
12                 Cabbagetown,St. James Town
13                 Cabbagetown,St. James Town
14                 Cabbagetown,St. James Town
15                 Cabbagetown,St. James Town
16                 Cabbagetown,St. James Town
17                 Cabbagetown,St. James Town
18                 Cabbagetown,St. James Town
19                 Cabbagetown,St. James Town
20                 Cabbagetown,St. James Town
21                 Cabbagetown,St.

## Analyze each neighborhood

In [59]:
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

In [60]:

downtown_onehot

,Adult Boutique,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,...,Thrift / Vintage Store,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
downtown_onehot['Neighborhood'] = downtown_venues['Neighborhood']

In [62]:
fixed_col = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])

In [63]:
downtown_onehot = downtown_onehot[fixed_col]

In [65]:
downtown_onehot.drop('Yoga Studio' , axis = 1, inplace=True)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [67]:
downtown_onehot

,Adult Boutique,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [68]:
downtown_onehot.shape

(1252, 184)

## Merge the details

In [103]:
downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index()

In [104]:
downtown_grouped

,Neighborhood,Adult Boutique,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide,King,Richmond",0.000000,0.000000,0.030000,0.010000,0.00,0.000000,0.030000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.016129,0.00,0.000000,0.000000,0.0,0.016129,...,0.000000,0.000000,0.000000,0.00,0.016129,0.000000,0.000000,0.000000,0.000000,0.000000
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.000000,0.014706,0.000000,0.00,0.000000,0.014706,0.0,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.014706,0.000000,0.000000,0.000000,0.000000
3,"Cabbagetown,St. James Town",0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.000000,0.010000,0.010000,0.01,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.010000,0.00,0.010000,0.000000,0.010000,0.010000,0.010000,0.000000
5,"Chinatown,Grange Park,Kensington Market",0.000000,0.000000,0.000000,0.031579,0.00,0.010526,0.000000,0.0,0.000000,...,0.000000,0.010526,0.010526,0.00,0.052632,0.000000,0.000000,0.042105,0.010526,0.000000
6,Christie,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Church and Wellesley,0.011236,0.011236,0.011236,0.000000,0.00,0.011236,0.000000,0.0,0.000000,...,0.011236,0.000000,0.000000,0.00,0.000000,0.000000,0.011236,0.011236,0.000000,0.011236
8,"Commerce Court,Victoria Hotel",0.000000,0.000000,0.040000,0.010000,0.00,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000
9,"Design Exchange,Toronto Dominion Centre",0.000000,0.000000,0.040000,0.010000,0.00,0.000000,0.010000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.01,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000


In [71]:
z = downtown_grouped[downtown_grouped['Neighborhood'] == 'Central Bay Street'].T.reset_index()

In [72]:
z.columns = ['Venue', 'Frequency']

In [73]:
z = z.iloc[1:,]

In [74]:
z = z.sort_values('Frequency' , ascending=False).reset_index(drop=True)

In [75]:
def split(row):
    try:
        category_list = row['Venue']
    except:
        category_list = row['venue.categories']
    if len(category_list) == 0:
        return None
    else:
        return (category_list.split('_')[1])


In [77]:
z.head()

,Venue,Frequency
0,Coffee Shop,0.08
1,Clothing Store,0.07
2,Cosmetics Shop,0.04
3,Plaza,0.03
4,Tea Room,0.03


In [81]:
downtown_grouped.head()

,Neighborhood,Adult Boutique,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide,King,Richmond",0.0,0.0,0.030000,0.010000,0.00,0.0,0.030000,0.0,0.000000,...,0.0,0.0,0.00,0.0,0.010000,0.000000,0.00,0.00,0.01,0.0
1,Berczy Park,0.0,0.0,0.000000,0.016129,0.00,0.0,0.000000,0.0,0.016129,...,0.0,0.0,0.00,0.0,0.016129,0.000000,0.00,0.00,0.00,0.0
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.0,0.0,0.014706,0.000000,0.00,0.0,0.014706,0.0,0.000000,...,0.0,0.0,0.00,0.0,0.000000,0.014706,0.00,0.00,0.00,0.0
3,"Cabbagetown,St. James Town",0.0,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.00,0.0,0.000000,0.000000,0.00,0.00,0.00,0.0
4,Central Bay Street,0.0,0.0,0.010000,0.010000,0.01,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.01,0.0,0.010000,0.000000,0.01,0.01,0.01,0.0


## Return common venues from Neighborhood

In [105]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [106]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Café,Coffee Shop,Hotel,Burger Joint,Bar,Restaurant,Steakhouse,Asian Restaurant,Gastropub,Deli / Bodega
1,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Seafood Restaurant,Steakhouse,Café,Farmers Market,Cheese Shop,Bakery,Breakfast Spot
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Coffee Shop,Italian Restaurant,Pub,Restaurant,Café,Gym / Fitness Center,Park,Sandwich Place,Bar,Bakery
3,"Cabbagetown,St. James Town",Coffee Shop,Restaurant,Park,Café,Bakery,Italian Restaurant,Pizza Place,Farmers Market,Farm,Butcher
4,Central Bay Street,Coffee Shop,Clothing Store,Cosmetics Shop,Tea Room,Plaza,Sushi Restaurant,Middle Eastern Restaurant,Fast Food Restaurant,Sporting Goods Shop,Chinese Restaurant
5,"Chinatown,Grange Park,Kensington Market",Café,Vegetarian / Vegan Restaurant,Bar,Vietnamese Restaurant,Mexican Restaurant,Dumpling Restaurant,Ice Cream Shop,Chinese Restaurant,Coffee Shop,Art Gallery
6,Christie,Café,Grocery Store,Playground,Italian Restaurant,Baby Store,Coffee Shop,Donut Shop,Farm,Falafel Restaurant,Exhibit
7,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Dance Studio,Gastropub,Fast Food Restaurant,Pub,Café
8,"Commerce Court,Victoria Hotel",Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Beer Bar,Steakhouse,Italian Restaurant,Seafood Restaurant,Gym
9,"Design Exchange,Toronto Dominion Centre",Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Deli / Bodega,Italian Restaurant,Gastropub,Beer Bar,Burger Joint


# Clustering

In [82]:
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 4, 1, 1, 1], dtype=int32)

In [83]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = downtown_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_merged.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.681940,-79.378474,3,Playground,Park,Bank,Tennis Court,Building,Donut Shop,Farmers Market,Farm,Falafel Restaurant,Exhibit
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.668160,-79.366602,1,Coffee Shop,Restaurant,Park,Café,Bakery,Italian Restaurant,Pizza Place,Farmers Market,Farm,Butcher
2,M4Y,Downtown Toronto,Church and Wellesley,43.666585,-79.381302,1,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Dance Studio,Gastropub,Fast Food Restaurant,Pub,Café
3,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.655120,-79.362640,0,Coffee Shop,Breakfast Spot,Restaurant,Spa,Thai Restaurant,Event Space,Mexican Restaurant,Electronics Store,Food Truck,Pub
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.657363,-79.378180,1,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Bar,Thai Restaurant,Japanese Restaurant,Italian Restaurant,Tea Room


In [89]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [93]:
map_clusters = folium.Map(location=[down_lat, down_lng], zoom_start=10)

In [94]:
map_clusters

## Maping clusters on Map

In [91]:

map_clusters = folium.Map(location=[down_lat, down_lng], zoom_start=10)


# add markers to the map

markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighborhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       

In [ ]:
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [95]:
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighborhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)



# Finish